# Info

This script analyzes TPD data. 

Uses:
- lmfit
- igor


-----------------------
Created by Jerry LaRue, larue@chapman.edu, 12/2018

Last modified by Jerry LaRue, larue@chapman.edu, 07/2020

# Select file

In [1]:
import numpy as np
import igor.igorpy as igor
import os
from os import listdir
from os.path import isfile, join
import sys
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import Button, Layout
from IPython.display import clear_output
import math
import struct
from importlib import reload
import plotly.express as px
from pandas import DataFrame as df

sys.path.append(os.getcwd() + '/Tools/')
import TDSTools
reload(TDSTools)

dt = TDSTools.DataTools()

ParameterFolder = os.getcwd()+'/TDS'

##### Widgets #####

ParameterFile = widgets.Dropdown(
    options=dt.FileList(ParameterFolder,['.yaml']),
    description='Select File',
    layout=Layout(width='70%'),
    style = {'description_width': '150px'},
    disabled=False,
)

display(ParameterFile)

##### Functions #####

def FileList() :
    return [f for f in listdir(FolderPath.value) if isfile(join(FolderPath.value, f))]

Dropdown(description='Select File', layout=Layout(width='70%'), options=('TPD181019_01 - CH3OH_O2_Ni', 'TPD181…

In [2]:
##### Load Data #####

tds = TDSTools.TDS((ParameterFolder,ParameterFile.value))
Data = tds.Data
Data = dt.TrimData(Data,115,2000)
Masses = tds.Parameters['Masses']

##### Functions #####

def SimulateTrace_Clicked(b) :
    with out :
        clear_output(True)
        
        try :
            tds.Parameters['HeatingRate'][0]
        except : 
            Rate = 60                 # K/min
        else :
            Rate = tds.Parameters['HeatingRate'][0]
        
        kB = 8.617e-5                 # eV/K
        IntRate = 0
        Ni = InitialCoverage.value
        Ea = DesorptionBarrier.value
        nu = Prefactor.value
        n = ReactionOrder.value
        yOffset = Offset.value

        Size = len(Data)

        Time = np.zeros((Size))
        Trace = np.zeros((Size))
        Coverage = np.zeros((Size))

        for idx, Temperature in enumerate(Data['Temperature (K)']) :
            Trace[idx] = nu*(Ni - IntRate)**n * np.exp(-Ea/(kB*Temperature))
            if idx != 0 :
                Time[idx] = PreviousTime + (Temperature - PreviousTemperature) / Rate
                IntRate += Trace[idx] * (Temperature - PreviousTemperature)
            Coverage[idx] = Ni - IntRate
            if IntRate >= Ni :
                IntRate = Ni
                Coverage[idx] = 0
            if Coverage[idx] < 0 or Coverage[idx] > Ni :
                Coverage[idx] = 0
                Trace[idx] = 0
            PreviousTime = Time[idx]
            PreviousTemperature = Temperature

        Data['Sim'] = Trace*ScalingFactor.value + yOffset
        fig = px.line(Data,x='Temperature (K)',y=Data.columns)
        fig.update_layout(yaxis_title='Intensity (au)',title_text=tds.Parameters['Description']+' with simulated trace',legend_title='',height=500)
        fig.show()

        CoverageSim = np.array((Data['Temperature (K)'].values,Coverage))
        CoverageSim = df(np.transpose(CoverageSim),columns=('Temperature (K)','Coverage'))
        fig = px.line(CoverageSim,x='Temperature (K)',y=CoverageSim.columns)
        fig.update_layout(yaxis_title='Coverage',showlegend=False,height=100)
        fig.show()

##### Widgets #####

InitialCoverage = widgets.BoundedFloatText(
    value=1,
    min=0,
    max=1,
    step=0.01,
    description='Initial Coverage',
    layout=Layout(width='30%'),
    style = {'description_width': '160px'},
    disabled=False
)

Offset = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=1000000,
    step=0.1,
    description='Offset',
    layout=Layout(width='30%'),
    style = {'description_width': '170px'},
    disabled=False
)

ScalingFactor = widgets.BoundedFloatText(
    value=10000000,
    min=0,
    max=1000000000,
    step=0.1,
    description='Scaling Factor',
    layout=Layout(width='30%'),
    style = {'description_width': '160px'},
    disabled=False
)

DesorptionBarrier = widgets.BoundedFloatText(
    value=0.7,
    min=0,
    max=10,
    step=0.01,
    description='Desorption Barrier (eV)',
    layout=Layout(width='30%'),
    style = {'description_width': '170px'},
    disabled=False
)

Prefactor = widgets.BoundedFloatText(
    value=1e6,
    min=0,
    max=1e10,
    step=1,
    description='Prefactor (Hz)',
    layout=Layout(width='30%'),
    style = {'description_width': '160px'},
    disabled=False
)

ReactionOrder = widgets.BoundedFloatText(
    value=1,
    min=1,
    max=2,
    step=1,
    description='Reaction Order',
    layout=Layout(width='30%'),
    style = {'description_width': '160px'},
    disabled=False
)

SimulateTrace = widgets.Button(description="Simulate Trace")
SimulateTrace.on_click(SimulateTrace_Clicked)

display(widgets.Box([InitialCoverage,Offset,ScalingFactor]))
display(widgets.Box([DesorptionBarrier,Prefactor,ReactionOrder]))

display(SimulateTrace)

out = widgets.Output()

##### Plot Data #####

with out :
    fig = px.line(Data,x='Temperature (K)',y=Data.columns)
    fig.update_layout(yaxis_title='Intensity (au)',title_text=tds.Parameters['Description'],legend_title='',height=500)
    fig.show()

out

Box(children=(BoundedFloatText(value=1.0, description='Initial Coverage', layout=Layout(width='30%'), max=1.0,…

Box(children=(BoundedFloatText(value=0.7, description='Desorption Barrier (eV)', layout=Layout(width='30%'), m…

Button(description='Simulate Trace', style=ButtonStyle())

Output()